In [10]:
%pip install sentence-transformers faiss-cpu numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
%pip install torch --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.6.0+cu118
True


In [3]:
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import torch
import json

d:\1P\SKU\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
import os
os.listdir('./content')

['clean_data.json', 'unclean_data.json']

In [17]:
import json

# Load the clean data file
with open('./content/clean_data.json', 'r') as f:
    clean_data = json.load(f)

# Load the unclean data file
with open('./content/unclean_data.json', 'r') as f:
    unclean_data = json.load(f)

# Check a sample of the data
print(list(clean_data.items())[:3])  # Display first 3 items of clean_data
print(list(unclean_data.items())[:3])  # Display first 3 items of unclean_data

[('135666', {'name': 'Avastin 100mg Injection', 'price': 33863, 'manufacturer_name': 'Roche Products India Pvt Ltd', 'pack_size_label': 'vial of 1 Injection', 'content': 'Bevacizumab (100mg)'}), ('227647', {'name': 'Actorise 40 Injection', 'price': 2928.2, 'manufacturer_name': 'Cipla Ltd', 'pack_size_label': 'prefilled syringe of 0.4 ml Injection', 'content': 'Darbepoetin alfa (40mcg)'}), ('228329', {'name': 'Actorise 25 Injection', 'price': 1815, 'manufacturer_name': 'Cipla Ltd', 'pack_size_label': 'prefilled syringe of 0.42 ml Injection', 'content': 'Darbepoetin alfa (25mcg)'})]
[('515278', {'c_barcode': '.', 'j_alternatives': [], 'c_hsn_code': '30049099', 'c_gst': '12.00', 'c_stock_availability': '0', 'j_molecules': [{'c_molecule_code': 'MC0265'}, {'c_molecule_code': 'MC0969', 'c_molecule_name': 'POVIDONE IODINE', 'c_description': 'Topical Antimicrobials, Antibacterial, Antibacterial Cleansing Agent, Antiseptic, Dressing Agent', 'c_usage': 'Topical solutionTopical ointmentTopical sw

In [4]:
# Initialize the SentenceTransformer model for generating embeddings on GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # Check if GPU is available
print(device)
model = SentenceTransformer('all-MiniLM-L12-v2', device=device)
# model = SentenceTransformer('all-mpnet-base-v2', device=device)

cuda


In [26]:
# Load the clean data (already loaded in your environment)
clean_data_subset = dict(list(clean_data.items())[:])

# Step 3: Extract relevant text from the clean data for embeddings
clean_texts = [
    # f"{item.get('name'),''} {item.get('content', '')} {item.get('manufacturer_name', '')} {item.get('pack_size_label', '')}"
    f"{item.get('name'),''} {item.get('manufacturer_name', '')}"
    for item in clean_data_subset.values()  # Use clean_data_subset here
]

index_file = "faiss_index_FlatL2-n-m.bin"

if os.path.exists(index_file):
    index = faiss.read_index(index_file)
    print("Loaded existing FAISS index...")
else:
    print("Creating new FAISS index...")
    # Generate embeddings for the clean data on GPU
    clean_embeddings = model.encode(clean_texts, convert_to_tensor=True, show_progress_bar=True)
    clean_embeddings = clean_embeddings.cpu().numpy()  # Move embeddings to CPU if necessary

    # Create a FAISS index
    dimension = clean_embeddings.shape[1]  # Embedding dimension (e.g., 384 for MiniLM)
    # index = faiss.IndexFlatIP(dimension, 10)  # Inner product search
    # index = faiss.IndexHNSWFlat(dimension, 10)  # Hierarchical Navigable Small World (HNSW)
    # index = faiss.IndexBinaryFlat(dimension)  # Binary Hamming distance
    index = faiss.IndexFlatL2(dimension)  # Exact nearest neighbor search(Euclidean Distance)

    # 10 is the number of neighbors for HNSW index

    # Add embeddings to the FAISS index
    index.add(clean_embeddings)

    # Save the FAISS index
    faiss.write_index(index, index_file)
    print("FAISS index saved.")


Creating new FAISS index...


Batches: 100%|██████████| 8783/8783 [02:58<00:00, 49.27it/s]


FAISS index saved.


In [34]:
# Extract unclean data (first 3 items)
unclean_data_subset = {k: unclean_data[k] for k in list(unclean_data.keys())[:10]}  # Extract first n items
# unclean_data_subset = unclean_data

# Step 7: Extract relevant text from the unclean data for embeddings
unclean_texts = [
    # f"{item['c_item_name']} {item['c_contains']} {item['c_mfg_name']}  {item['n_pack_size']} {item['c_pack_type_name']}"
    f"{item['c_item_name']} {item['c_item_name']} {item['c_item_name']} {item['c_mfg_name']}  {item['c_mfg_name']}"
    # weights in 3:2 (name and mfg name)
    for item in unclean_data_subset.values()
]

# Generate embeddings for the unclean data
unclean_embeddings = model.encode(unclean_texts, convert_to_tensor=True)
unclean_embeddings = unclean_embeddings.cpu().numpy()

# Perform similarity search for each unclean item
k = 10  # Number of closest matches
distances, indices = index.search(unclean_embeddings, k)

# Print the results
for i, unclean_item in enumerate(unclean_data_subset.values()):  # Use unclean_data_subset here
    print(f"Unclean Item: {unclean_item['c_item_name']} -- {unclean_item['c_mfg_name']}")
    print(f"Top {k} Closest Matches:")
    for j in range(k):
        # Ensure indices are valid for clean_data_subset
        closest_match = list(clean_data_subset.values())[indices[i][j]]
        print(f"  {closest_match} (Distance: {distances[i][j]})")
    print("-" * 50)


Unclean Item: MOSS FRESH EYE DROPS -- SYNTHO PHARMA CHEMICALS
Top 10 Closest Matches:
  {'name': 'Moss Fresh Gel Eye Drop', 'price': 165, 'manufacturer_name': 'Syntho Pharmaceuticals Pvt Ltd', 'pack_size_label': 'bottle of 10 ml Eye Drop', 'content': 'Carboxymethylcellulose (1% w/v)'} (Distance: 0.6039383411407471)
  {'name': 'Eyewet Eye Drop', 'price': 96, 'manufacturer_name': 'Medimind Drugs and Chemicals', 'pack_size_label': 'bottle of 10 ml Ophthalmic Solution', 'content': 'Carboxymethylcellulose (0.5% w/v)'} (Distance: 0.6350395679473877)
  {'name': 'Flucol Eye Drop', 'price': 25.02, 'manufacturer_name': 'SKN Organics', 'pack_size_label': 'packet of 15 ml Eye Drop', 'content': 'Fluconazole (0.3% w/v)'} (Distance: 0.703702449798584)
  {'name': 'Eslate Eye Drops', 'price': 100, 'manufacturer_name': 'Essential Pharmaceuticals', 'pack_size_label': 'bottle of 5 ml Ophthalmic Solution', 'content': 'Cyclopentolate (1% w/v)'} (Distance: 0.7104870676994324)
  {'name': 'Moss Plus Eye Drop',

In [41]:
output_file = "matches.json"

# Initialize or load existing data
if os.path.exists(output_file):
    with open(output_file, "r") as f:
        try:
            results = json.load(f)
        except json.JSONDecodeError:
            results = []
else:
    results = []

for i, (item_id, unclean_item) in enumerate(unclean_data_subset.items()):
    match_data = {
        "unclean_item": {
            "id": item_id,
            "name": unclean_item.get("c_item_name", ""),
            "manufacturer": unclean_item.get("c_mfg_name", "")
        },
        "matches": []
    }


    for j in range(k):
        closest_match = list(clean_data_subset.values())[indices[i][j]]
        match_data["matches"].append({
            "id": list(clean_data_subset.keys())[indices[i][j]],  # Get the corresponding key (ID)
            "match": closest_match,
            "distance": float(distances[i][j])  # Convert np.float32 to Python float
        })


    results.append(match_data)

    # Save to file after each iteration
    with open(output_file, "w") as f:
        json.dump(results, f, indent=4)
